In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
# from config import api_key

## base_url = https://api.census.gov/data/2019/acs/acs5/subject?get=NAME,S0101_C01_001E&for=region:*&key=api_key

api_key = "0e395bcf9cca78932f9c8e80c0a8a55fe1cba723"
# Census API Key

c = Census(api_key, year=2019)

## "0e395bcf9cca78932f9c8e80c0a8a55fe1cba723"

# STATE CODES:
# 04|AZ|Arizona|01779777
# 06|CA|California|01779778
# 08|CO|Colorado|01779779
# 16|ID|Idaho|01779783
# 30|MT|Montana|00767982
# 32|NV|Nevada|01779793
# 35|NM|New Mexico|00897535
# 41|OR|Oregon|01155107
# 49|UT|Utah|01455989
# 53|WA|Washington|01779804
# 56|WY|Wyoming|01779807


In [34]:
census_data = c.acs5.get(("NAME", "B01003_001E", "B19013_001E", "B01003_001E", "B01002_001E", "B11001_001E",
                          "B19301_001E", "B08303_001E", "B08015_001E",
                          "B17001_002E"), {'for': 'zip code tabulation area:*'})

# # Convert to DataFrame
census_pd = pd.DataFrame(census_data)
census_pd.head()


,NAME,B01003_001E,B19013_001E,B01002_001E,B11001_001E,B19301_001E,B08303_001E,B08015_001E,B17001_002E,state,zip code tabulation area
0,ZCTA5 00601,17113.0,14361.0,41.9,5509.0,7493.0,3504.0,3115.0,10552.0,72,00601
1,ZCTA5 00602,37751.0,16807.0,42.9,12740.0,9694.0,10525.0,8645.0,18653.0,72,00602
2,ZCTA5 00603,47081.0,16049.0,42.1,19228.0,11259.0,11479.0,10460.0,23691.0,72,00603
3,ZCTA5 00606,6392.0,12119.0,44.3,1946.0,6093.0,1228.0,1085.0,4185.0,72,00606
4,ZCTA5 00610,26686.0,19898.0,42.7,8795.0,10572.0,8065.0,6800.0,12204.0,72,00610


In [30]:
# Column Reordering
census_pd = census_pd.rename(columns={"NAME": "State ID",
                                      "State": "State ID",
                                      "B01003_001E": "Population",
                                      "B11001_001E": "Number of Households",
                                      "B19013_001E": "Household Income",
#                                       "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B08303_001E": "Total Commute Time",
                                      "B08015_001E": "Total Vehicles",
                                      "NAME": "Name", "zip code tabulation area": "Zipcode"})



census_pd.head()


,State ID,Zipcode,Population,Number of Households,Household Income,Poverty Count,Poverty Rate,Total Commute Time,Average Commute Time,Total Vehicles,Average Vehicles per Household
0,72,00601,17113.0,5509.0,14361.0,10552.0,61.660726,3504.0,0.636050,3115.0,0.565438
1,72,00602,37751.0,12740.0,16807.0,18653.0,49.410612,10525.0,0.826138,8645.0,0.678571
2,72,00603,47081.0,19228.0,16049.0,23691.0,50.319662,11479.0,0.596994,10460.0,0.543998
3,72,00606,6392.0,1946.0,12119.0,4185.0,65.472466,1228.0,0.631038,1085.0,0.557554
4,72,00610,26686.0,8795.0,19898.0,12204.0,45.731844,8065.0,0.916998,6800.0,0.773167


In [5]:
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)

census_pd["Average Vehicles per Household"] = census_pd["Total Vehicles"] / census_pd["Number of Households"]

census_pd["Average Commute Time"] = census_pd["Total Commute Time"] / census_pd["Number of Households"]

census_pd = census_pd[["State ID", "Zipcode", "Population", "Number of Households", 
                       "Household Income", "Poverty Count", "Poverty Rate", 
                       "Total Commute Time", "Average Commute Time",
                       "Total Vehicles", "Average Vehicles per Household"]]

# Visualize
print(len(census_pd))
census_pd.head()
    

33120


,State,Zipcode,Population,Number of Households,Household Income,Poverty Count,Poverty Rate,Total Commute Time,Average Commute Time,Total Vehicles,Average Vehicles per Household
0,72,00601,17113.0,5509.0,14361.0,10552.0,61.660726,3504.0,0.636050,3115.0,0.565438
1,72,00602,37751.0,12740.0,16807.0,18653.0,49.410612,10525.0,0.826138,8645.0,0.678571
2,72,00603,47081.0,19228.0,16049.0,23691.0,50.319662,11479.0,0.596994,10460.0,0.543998
3,72,00606,6392.0,1946.0,12119.0,4185.0,65.472466,1228.0,0.631038,1085.0,0.557554
4,72,00610,26686.0,8795.0,19898.0,12204.0,45.731844,8065.0,0.916998,6800.0,0.773167


In [7]:
census_pd.to_csv("census_data.csv", encoding="utf-8", index=False)

In [32]:
states = ["04", "06", "08", "16", "30", "32", "35", "41", "49", "53", "56"]

state_name = {"04": "AZ",
             "06": "CA",
             "08": "CO",
             "16": "ID",
             "30": "MT",
             "32": "NV",
             "35": "NM",
             "41": "OR",
             "49": "UT",
             "53": "WA",
             "56": "WY"}

state_census_df = census_pd.loc[census_pd["State ID"].isin (states)]
state_census_df

state_census_df["State"] = [state_name[state] for state in state_census_df["State ID"]]
state_census_df



<ipython-input-32-6c82f8bf2b60>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_census_df["State"] = [state_name[state] for state in state_census_df["State ID"]]


,State ID,Zipcode,Population,Number of Households,Household Income,Poverty Count,Poverty Rate,Total Commute Time,Average Commute Time,Total Vehicles,Average Vehicles per Household,State
20057,30,59001,1558.0,734.0,58482.0,14.0,0.898588,679.0,0.925068,545.0,0.742507,MT
20058,30,59002,127.0,47.0,37917.0,33.0,25.984252,52.0,1.106383,45.0,0.957447,MT
20059,30,59003,1359.0,433.0,46838.0,362.0,26.637233,490.0,1.131640,325.0,0.750577,MT
20060,30,59006,1461.0,518.0,44750.0,377.0,25.804244,543.0,1.048263,485.0,0.936293,MT
20061,30,59007,172.0,66.0,105625.0,13.0,7.558140,72.0,1.090909,65.0,0.984848,MT
...,...,...,...,...,...,...,...,...,...,...,...,...
32877,53,99363,350.0,87.0,38011.0,298.0,85.142857,78.0,0.896552,NaN,NaN,WA
32878,53,99371,325.0,132.0,53750.0,54.0,16.615385,113.0,0.856061,100.0,0.757576,WA
32879,53,99401,642.0,176.0,67917.0,55.0,8.566978,189.0,1.073864,NaN,NaN,WA
32880,53,99402,1325.0,608.0,55595.0,84.0,6.339623,596.0,0.980263,510.0,0.838816,WA


In [36]:
state_census_df.to_csv("state_census_df.csv", encoding="utf-8", index=False)